In [ ]:
# Import the required modules
from IPython.display import clear_output
import PIL.Image
from io import BytesIO
import IPython.display

import cv2
import time
%pylab inline

In [ ]:
DEBUG = False

In [ ]:
# UTILITY FUNCTIONS

In [ ]:
#Use 'jpeg' instead of 'png' (~5 times faster)
def showarray(a, times=2, r=True, fmt='jpeg'):
    old_shape = a.shape
    if len(old_shape) is 3:
        a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
        old_shape = a.shape[:-1]
    if r:
        new_shape = flip(rint(divide(old_shape, times)).astype(int))
        a = cv2.resize(a, tuple(new_shape.tolist()))
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [ ]:
def get_frame(cam):
    # Capture frame-by-frame
    ret, frame = cam.read()
    if not ret:
        raise KeyboardInterrupt()
   
    return frame

In [ ]:
yellow = (0, 255, 255)
red = (0, 0, 255)
green = (0, 255, 0)
white = (255,0,0)
def draw_region(frame, h, w, color, l=None, r=None):
    frame = cv2.line(frame, (2000, h), (0, h), color, 5)
    frame = cv2.line(frame, (2000, h-w), (0, h-w), color, 5)
    if l is not None:
        frame = cv2.line(frame, (l, 2000), (l, 0), color, 5)
        frame = cv2.line(frame, (r, 2000), (r, 0), color, 5)
    return frame

In [ ]:
# PROCESS FUNCTIONS

In [ ]:
def generate_LUT(gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    def adjust_gamma(image):
        # apply gamma correction using the lookup table
        return cv2.LUT(image, table)
    return adjust_gamma

In [ ]:
def DoG(frame, s=1, stride=0):
    #run a 5x5 gaussian blur then a 3x3 gaussian blr
    sB = (s+1+stride)*2+1
    sS = (s)*2+1
    blurB = cv2.GaussianBlur(frame,(sB,sB),0)
    blurS = cv2.GaussianBlur(frame,(sS,sS),0)

    return blurB - blurS

In [ ]:
def generate_morphOps():
    se_c = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    se_vl = np.array([[0,1,0],[0,1,0],[0,1,0]], np.ubyte)
    def morphOps(frame, it):
        frame_out = cv2.morphologyEx(frame, cv2.MORPH_OPEN, se_c, iterations=1)
        frame_out = cv2.morphologyEx(frame_out, cv2.MORPH_OPEN, se_vl, iterations=it)
        frame_out = cv2.morphologyEx(frame_out, cv2.MORPH_DILATE, se_vl, iterations=it)
        return frame_out
    return morphOps

In [ ]:
def Unwarp(frame, ltc, rtc):
    h = frame.shape[0]
    w = frame.shape[1]
    pts_src = np.array([[ltc, 0], [0, h], [rtc, h],[w, 0]])
    pts_dst = np.array([[0, 0],[0, h],[w, h],[w, 0]])

    h, status = cv2.findHomography(pts_src, pts_dst)
    frame_out = cv2.warpPerspective(frame, h, (frame.shape[1],frame.shape[0]))
    return frame_out

In [ ]:
#Kartashev: interpolation and gluing
from scipy.interpolate import UnivariateSpline

def glue(sum_plate, region):
    old_indices = arange(0, region.shape[1])
    new_indices = np.linspace(0, region.shape[1]-1, sum_plate.shape[1])
    for line_i in range(region.shape[0]):
        line = region[line_i,:]
        spl = UnivariateSpline(old_indices, line, k=3, s=0)
        new_line = spl(new_indices).astype(np.uint8)
        sum_plate = np.vstack((sum_plate, [new_line]))
    return sum_plate

In [ ]:
def generate_process(scan_line_h, scan_line_w):
    adjust_gamma = generate_LUT(4)
    morphOps = generate_morphOps()
    def process(frame_in):
        frame_in = cv2.cvtColor(frame_in, cv2.COLOR_BGR2GRAY)
        frame_out = cv2.equalizeHist(frame_in)
        frame_out = adjust_gamma(frame_out)
        frame_out = DoG(frame_out, 2, 9)
        frame_out = morphOps(frame_out, 6)
        
        frame_out = Unwarp(frame_out, 140, frame_out.shape[1]+160)
        frame_in = Unwarp(frame_in, 140, frame_out.shape[1]+160)
        
        lcm = 220
        rcm = frame_out.shape[1]-310
        frame_out = frame_out[:, lcm:rcm]
        frame_in = frame_in[:, lcm:rcm]
        
        region_sum_t = sum(sum(frame_out[scan_line_h-scan_line_w:scan_line_h-int(scan_line_w/2),:]))/frame_out.size
        region_sum_b = sum(sum(frame_out[scan_line_h-int(scan_line_w/2):scan_line_h,:]))/frame_out.size
        if DEBUG:
            print(region_sum_t, region_sum_b)
            global_sum.append(region_sum_b)
            global_sum.append(region_sum_t)
        frame_out = frame_in
        
        # Грязь по кромке, годные, изломы 0.5
        # Годные 0.8 
        threshold = 0.8
        if region_sum_t < threshold or region_sum_b < threshold:
            scanned_line = frame_in[scan_line_h-scan_line_w:scan_line_h,:]
            return frame_in, scanned_line
        else:
            return frame_in, None

    return process

In [ ]:
# Work area

In [ ]:
input_file_path = 'Годные.cva'
vid = cv2.VideoCapture(input_file_path)

In [ ]:
# Seek needed frame
vid.set(cv2.CAP_PROP_POS_FRAMES,150)
frame = get_frame(vid)

In [ ]:
scan_line_h = 450
scan_line_w = 105
process = generate_process(scan_line_h, scan_line_w)
found_plate = None
sum_plate = None
plates_count = 0
frames_to_play = 50
status = ''
if DEBUG:
    global_sum = []
try:
    while(frames_to_play):
        t1 = time.time()
        
        # Capture frame-by-frame
        frame = get_frame(vid)
        #frames_to_play -= 1

        # Convert the image from OpenCV BGR format to matplotlib RGB format
        # to display the image
        frame_p, line = process(frame)        
        if DEBUG:
            frame_show = frame_p.copy()
            if line is not None:
                showarray(line)
        if line is not None:
            if not found_plate:
                status = 'plate have started'
                found_plate = True
                sum_plate = line
            else:
                #пока отображаем только последнюю. И в Grayscale, так проще делать интерполяцию                
                sum_plate = glue(sum_plate, line)
                status = 'appended plate'
        else:
            if found_plate:
                status = 'plate has ended'
                found_plate = False
                if sum_plate.shape[0] > 900:
                    cv2.imwrite('plate_%d.tiff' % (plates_count),sum_plate)
                    plates_count += 1
            else:
                status = 'No plate'
        t2 = time.time()
        if DEBUG:
            showarray(draw_region(frame_show, scan_line_h, scan_line_w, white), 4)
            time.sleep(1)

        fps = '%f FPS' % (1/(t2-t1+1e-07))
        print('%s %s' % (status, fps))
        # Display the frame until new frame is available
        clear_output(wait=True)
except KeyboardInterrupt:
    vid.release()
    print('Stream stopped')
#showarray(sum_plate, 4)

In [ ]:
# TEST

In [ ]:
# Select threshold

In [ ]:
global_sum = np.array(global_sum, dtype=float64)

In [ ]:
hist(global_sum, 40);

In [ ]:
plot(sort(global_sum));

In [ ]:
plot(global_sum_rate)

In [ ]:
plot(global_sum_rate > 0.015);

In [ ]:
global_sum_rate = diff(sort(global_sum))
global_threshold = sort(global_sum)[argmax(global_sum_rate > 0.015)]
global_threshold